In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import lightning as L
import torch


from SoccerNet.Evaluation.MV_FoulRecognition import evaluate
from torch.utils.data import DataLoader, random_split

from src.training import TrainingConfig
from src.dataset import MultiViewDataset
from src.model import LitMVNNetwork, get_pre_model
from src.loss import get_criterion
from src.eval import save_evaluation_file
from datetime import datetime
import os
from src.augment import get_augmentation
import wandb
from lightning.pytorch.loggers import WandbLogger

In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: adamsebastiangorski (wut-zzsn). Use `wandb login --relogin` to force relogin


True

In [3]:
torch.cuda.empty_cache()

In [4]:
num_epochs = 5
start_frame = 0
end_frame = 100
fps = 25
num_views = 5
pre_model = "s3d"
max_num_worker = 0
batch_size = 2
data_aug = False
pooling_type = 'max'
weight_decay = 0.001
step_size = 3
gamma = 0.1
LR = 0.01
weighted_loss = False
data_aug = False

training_config = TrainingConfig(start_frame=start_frame, end_frame=end_frame, fps=fps, num_views = num_views, pre_model = pre_model,
                                 max_num_worker=max_num_worker, batch_size=batch_size, data_aug=data_aug, pooling_type=pooling_type, 
                                 weight_decay=weight_decay, step_size=step_size, gamma=gamma, LR=LR, weighted_loss=weighted_loss)


In [5]:
wandb.init(
    project="ZZSN multi-view-foul-recognition",
    config=training_config.model_dump()
)

In [8]:
path = "/net/tscratch/people/plgmiloszl/data"
predictions_output_dir = "outputs"

transform_aug = get_augmentation(training_config.data_aug)
transforms_model = get_pre_model(training_config.pre_model)


In [9]:
dataset_Train = MultiViewDataset(path=path, start=start_frame, end=end_frame, fps=fps, split='Train',
                                 num_views=5, transform=transform_aug, transform_model=transforms_model)

train_size = int(0.7 * len(dataset_Train))
val_size = len(dataset_Train) - train_size

train_set, val_set = random_split(dataset_Train, [train_size, val_size])
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=max_num_worker, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=max_num_worker, pin_memory=True) 

2319


In [10]:
dataset_dir = "datasets"

dataset_Test = MultiViewDataset(path=path, start=start_frame, end=end_frame, fps=fps, split='Test', num_views = 5, 
transform_model=transforms_model)


dataset_Chall = MultiViewDataset(path=path, start=start_frame, end=end_frame, fps=fps, split='Chall', num_views = 5, 
        transform_model=transforms_model)

test_loader = torch.utils.data.DataLoader(dataset_Test,
            batch_size=1, shuffle=False,
            num_workers=max_num_worker, pin_memory=True)
        
chall_loader = torch.utils.data.DataLoader(dataset_Chall,
            batch_size=1, shuffle=False,
            num_workers=max_num_worker, pin_memory=True)

251
273


In [11]:
criterion = get_criterion(weighted_loss, dataset_train=dataset_Train)
model = LitMVNNetwork(pre_model=pre_model, pooling_type=pooling_type, criterion=criterion, config=training_config)
job_id = str(datetime.now())
wand_logger = WandbLogger(log_model="all")

Downloading: "https://download.pytorch.org/models/s3d-d76dad2f.pth" to /net/people/plgrid/plgmiloszl/.cache/torch/hub/checkpoints/s3d-d76dad2f.pth
100%|██████████| 32.0M/32.0M [00:00<00:00, 125MB/s] 


In [12]:
trainer = L.Trainer(limit_train_batches=3000, max_epochs=num_epochs, logger=wand_logger)
trainer.fit(model=model, train_dataloaders=train_loader)

/net/tscratch/people/plgmiloszl/.venv/lib64/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.9 /net/tscratch/people/plgmiloszl/.venv/lib64/pytho ...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/net/tscratch/people/plgmiloszl/.venv/lib64/python3.9/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/net/tscratch/people/plgmiloszl/.venv/lib64/python3.9/site-packages/lightning/pytorch/loggers/wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLog

Epoch 0:   0%|          | 0/812 [00:00<?, ?it/s] 

/net/tscratch/people/plgmiloszl/.venv/lib64/python3.9/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


RuntimeError: [enforce fail at alloc_cpu.cpp:117] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 1284505600 bytes. Error code 12 (Cannot allocate memory)

In [12]:
os.makedirs(predictions_output_dir, exist_ok=True)

test_set = f"test_{job_id}"
chall_set = f"chall_{job_id}"

test_prediction_file = save_evaluation_file(test_loader, model=model, set_name=test_set, output_dir=predictions_output_dir)
chall_prediction_file = save_evaluation_file(chall_loader, model=model, set_name=chall_set, output_dir=predictions_output_dir)

100%|██████████| 273/273 [13:22<00:00,  2.94s/it]


In [15]:
test_results = evaluate(os.path.join(path, "Test", "annotations.json"), test_prediction_file)
wandb.log(test_results)

<class 'numpy.ndarray'>


Error: You must call wandb.init() before wandb.log()

In [16]:
wandb.finish()